In [7]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

import rasterio
from osgeo import gdal
import geopandas as gpd

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image




In [2]:
shp = gpd.read_file('../data/building_footprints/building_footprint_roi_challenge.shp')

In [ ]:
shp.head()


In [ ]:
# Plot the building footprints
fig, ax = plt.subplots(figsize=(10, 10))  # Adjust the figure size as needed
shp.plot(ax=ax, color='blue', edgecolor='black')  # You can customize colors

plt.title('Building Footprints')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


In [ ]:
from shapely.geometry import box

post_event_image ='../data/raw/Post_Event_San_Juan.tif'
with rasterio.open(post_event_image) as src:
    bounds = src.bounds
    crs = src.crs

# Create a Polygon from the bounds
tiff_polygon = box(bounds.left, bounds.bottom, bounds.right, bounds.top)

# Convert the Polygon into a GeoDataFrame
tiff_gdf = gpd.GeoDataFrame({'geometry': [tiff_polygon]}, crs=crs)



# Ensure the building footprints are in the same CRS as the TIFF file
building_footprints_gdf = shp.to_crs(crs)

intersects = building_footprints_gdf.intersects(tiff_polygon)
print(sum(intersects))
print(len(building_footprints_gdf))
# Plotting
# fig, ax = plt.subplots(figsize=(10, 10))
# tiff_gdf.plot(ax=ax, facecolor="none", edgecolor='red', linewidth=2, label='TIFF Extent')
# building_footprints_gdf.plot(ax=ax, color='blue', edgecolor='black', alpha=0.5, label='Building Footprints')

# plt.title('TIFF Extent and Building Footprints')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.legend()
# plt.show()

In [ ]:
from osgeo import gdal, osr, ogr

# Open the GeoTIFF file
tif_ds = gdal.Open(post_event_image, gdal.GA_ReadOnly)

# Get the spatial reference
tif_srs = tif_ds.GetProjectionRef()

# Print the WKT SRS
print("GeoTIFF WKT SRS:\n", tif_srs)

# Open the shapefile
shp_file_path = '/home/projects/storm-damage/data/building_footprints/building_footprint_roi_challenge.shp'
new_shp_file_path = '/home/projects/storm-damage/data/building_footprints/building_footprint_reprojected.shp'
shp_ds = ogr.Open(shp_file_path, 0)  # 0 means read-only

crs_utm_19N = 'EPSG:32619'
gdf_reprojected = shp.to_crs(crs_utm_19N)
# shp_layer = shp_ds.GetLayer()

# # Fetch the original SRS
# shp_srs = shp_layer.GetSpatialRef()

# # Print the original SRS WKT
# print("Original Shapefile WKT SRS:\n", shp_srs.ExportToWkt())

# # Target SRS from the GeoTIFF WKT
# target_srs = osr.SpatialReference()
# target_srs.ImportFromWkt(tif_srs)

# # Reproject the shapefile
# # For reprojection, we'll use a command-line call to ogr2ogr. This is an example to run outside Python.
# # Replace 'path/to/your/new_shapefile.shp' with the desired output path for the reprojected shapefile
# ogr2ogr_command = f'ogr2ogr -t_srs "{tif_srs}" "{new_shp_file_path}" "{shp_file_path}"'
# print("Run this command in your terminal:\n", ogr2ogr_command)

In [ ]:
import rasterio
import geopandas as gpd
from shapely.geometry import box
from osgeo import gdal

# Assuming post_event_image and gdf_reprojected are defined as per your setup
post_event_image = '../data/raw/Post_Event_San_Juan.tif'

# Open the TIFF file to get its geotransform and CRS
with rasterio.open(post_event_image) as src:
    bounds = src.bounds
    crs = src.crs

# Define the size of your grid/tiles
grid_x, grid_y = 512, 512  # Example sizes
tile_x, tile_y = 73, 102  # Example tile position

# Calculate the bounds for the specified grid
minx = bounds.left + tile_x * grid_x * src.res[0]  # Adjusted for tile_x position
maxy = bounds.top - tile_y * grid_y * abs(src.res[1])  # Adjusted for tile_y position
miny = maxy - grid_y * abs(src.res[1])  # Adjusted for height of the tile
maxx = minx + grid_x * src.res[0]  # Adjusted for width of the tile

# Create a polygon for the specified tile
first_tile_polygon = box(minx, miny, maxx, maxy)

# Convert the polygon into a GeoDataFrame
first_tile_gdf = gpd.GeoDataFrame({'geometry': [first_tile_polygon]}, crs=crs)

# Ensure the building footprints are in the same CRS as the first_tile_gdf
building_footprints_gdf = gdf_reprojected.to_crs(first_tile_gdf.crs)

# Check if the specified tile intersects with any building footprint
intersects = building_footprints_gdf.intersects(first_tile_polygon)

# If any intersection occurs, this will be True
intersection_exists = intersects.any()

print(f"Does tile {tile_x, tile_y} intersect with any building footprint? {intersection_exists}")


In [ ]:
print(gdf_reprojected.head())
print(shp.head())

In [3]:
import os
import geopandas as gpd
import rasterio
from shapely.geometry import Polygon, box
from osgeo import gdal, osr

def generate_tiles(input_file, output_dir, grid_x, grid_y, shp):
    # Open the source raster file
    with rasterio.open(input_file) as src:
        width, height = src.width, src.height
        crs = src.crs
        bounds = src.bounds
    
    building_footprints_gdf = shp.to_crs(crs)
    ds = gdal.Open(input_file)

    # Get image size and number of bands
    width = ds.RasterXSize
    height = ds.RasterYSize
    num_bands = ds.RasterCount

    # Calculate number of tiles in each dimension
    num_tiles_x = (width // grid_x)
    num_tiles_y = (height // grid_y)

    print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)
    print('here')
    count = 0
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            x_offset, y_offset = i * grid_x, j * grid_y
            minx = bounds.left + i * grid_x * src.res[0]  # Adjusted for tile_x position
            maxy = bounds.top - j * grid_y * abs(src.res[1])  # Adjusted for tile_y position
            miny = maxy - grid_y * abs(src.res[1])  # Adjusted for height of the tile
            maxx = minx + grid_x * src.res[0]  # Adjusted for width of the tile

            # Create a bounding box for the tile
            tile_bbox = box(minx, miny, maxx, maxy)
            tile_width = min(grid_x, width - x_offset)
            tile_height = min(grid_y, height - y_offset)
            # Check for intersection with building footprints
            intersects = building_footprints_gdf.intersects(tile_bbox).any()
            
            if intersects:
                count += 1
                print(f"Tile {i}, {j} intersects with building footprint(s).")

                # Read the tile data
                tile = []
                for band in range(1, num_bands + 1):
                    tile_data = ds.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
                    tile.append(tile_data)
                # Save the tile using GDAL
                output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")
                driver = gdal.GetDriverByName('GTiff')
                options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
                out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
                       ds.GetRasterBand(1).DataType, options=options)
                
                
                geotransform = list(ds.GetGeoTransform())
                geotransform[0] = geotransform[0] + x_offset * geotransform[1]
                geotransform[3] = geotransform[3] + y_offset * geotransform[5]
                out_ds.SetGeoTransform(tuple(geotransform))

                # Set the projection
                out_ds.SetProjection(ds.GetProjection())

                # Write each band to the output file
                for band in range(1, num_bands + 1):
                    out_band = out_ds.GetRasterBand(band)
                    out_band.WriteArray(tile[band - 1])

                # Close the output file
                out_ds = None

# Example usage
# input_file = 'path_to_your_tiff.tif'
# output_dir = 'path_to_output_directory'
# grid_x, grid_y = 512, 512
# building_footprints_gdf = your_building_footprints_gdf_loaded_and_reprojected_if_necessary
# generate_tiles(input_file, output_dir, grid_x, grid_y, building_footprints_gdf)


In [4]:
input_file = "../data/raw/Post_Event_San_Juan.tif"
output_dir = "../data/intermediate/Post_Event_Grids_In_TIFF_all_buildings"
grid_x = 512
grid_y = 512
generate_tiles(input_file, output_dir, grid_x, grid_y, shp)


Total number of tiles: 10730
here
Tile 0, 11 intersects with building footprint(s).
Tile 0, 12 intersects with building footprint(s).
Tile 0, 13 intersects with building footprint(s).
Tile 0, 14 intersects with building footprint(s).
Tile 0, 19 intersects with building footprint(s).
Tile 0, 20 intersects with building footprint(s).
Tile 0, 21 intersects with building footprint(s).
Tile 0, 22 intersects with building footprint(s).
Tile 0, 23 intersects with building footprint(s).
Tile 0, 24 intersects with building footprint(s).
Tile 0, 26 intersects with building footprint(s).
Tile 0, 27 intersects with building footprint(s).
Tile 0, 28 intersects with building footprint(s).
Tile 0, 29 intersects with building footprint(s).
Tile 0, 30 intersects with building footprint(s).
Tile 0, 31 intersects with building footprint(s).
Tile 0, 32 intersects with building footprint(s).
Tile 0, 33 intersects with building footprint(s).
Tile 0, 35 intersects with building footprint(s).
Tile 0, 36 inter

In [ ]:
building_footprints_gdf.head()

In [5]:
def convert_tiff_to_jpeg(input_dir,output_dir):
    # check if output_dir exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        # check if file is an image (ends with .tif)
        if filename.endswith('.tif'):
            img = Image.open(os.path.join(input_dir, filename))
        
            # check if image is RGB mode, if not convert it
            if img.mode != 'RGB':
                img = img.convert('RGB')
        
            # create new filename, replace .tif with .jpg
            output_filename = os.path.splitext(filename)[0] + '.jpg'
        
            # save the image in JPEG format
            img.save(os.path.join(output_dir, output_filename), 'JPEG')
    print("Conversion from TIFF to JPEG completed.")

In [8]:
input_dir = "../data/intermediate/Pre_Event_Grids_In_TIFF_all_buildings"
output_dir = "../data/intermediate/Pre_Event_Grids_In_JPEG_all_buildings"
convert_tiff_to_jpeg(input_dir,output_dir)

Conversion from TIFF to JPEG completed.
